# Example usage of xTB with XYZ files

You can still use molobjs (or ppqm functions) directly, if you have XYZ files to compute.


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import logging
import sys

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem.Draw import MolsToGridImage

In [ ]:
try:
    import ppqm
except ModuleNotFoundError:
    import pathlib

    cwd = pathlib.Path().resolve().parent
    sys.path.append(str(cwd))
    import ppqm

In [ ]:
from ppqm import jupyter as ppqm_jupyter

In [ ]:
import rmsd

## Set logging level

In [ ]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger("ppqm").setLevel(logging.INFO)
logging.getLogger("xtb").setLevel(logging.DEBUG)
show_progress = False

## Load a XYZ file

In [ ]:
filename = "../tests/resources/compounds/CHEMBL1234757.xyz"

In [ ]:
atoms, coordinates = rmsd.get_coordinates_xyz(filename)

In [ ]:
atoms

In [ ]:
charge = 0

## View the molecule

Note that no explict bonds are written in XYZ format, only the coordinates.
So RDKit does not now how the graph of the molecule looks like


In [ ]:
molobj = ppqm.chembridge.axyzc_to_molobj(atoms, coordinates, charge)
molobj

In [ ]:
ppqm_jupyter.show_molobj(molobj)

## Set XTB Settings

In [ ]:
xtb_options = {
    "scr": "./_tmp_directory_",  # Where should the calculations happen?
    "cmd": "xtb",  # What is the executable?
    "n_cores": 1,  # How many cores to use?
    "show_progress": show_progress,
}

## Calculate some properties

We can now define calculation options and get XTB properties for each conformer in the molobj.

The options for XTB follows the --arg documentation from the xtb website https://xtb-docs.readthedocs.io/en/latest/commandline.html


In [ ]:
# Optimize molecule in water
optimize_options = {
    "gfn": 2,
    "alpb": "h2o",
    "opt": None,
}

Optimize molobj and set resulting coordinates. The `.calculate` function will return a `List[Dict]` with a property dictionary for each conformer.

In [ ]:
molecule_properties = ppqm.xtb.get_properties_from_axyzc(
    atoms, coordinates, charge, options=optimize_options, **xtb_options
)

In [ ]:
list(molecule_properties.keys())

In [ ]:
energy = molecule_properties["scc_energy"]
energy *= ppqm.units.hartree_to_kcalmol
print(f"The energy was {energy:.2f} kcal/mol")

In [ ]:
new_coordinates = molecule_properties["coord"]

In [ ]:
diff = rmsd.kabsch_rmsd(coordinates, new_coordinates)
print(f"The structure moved {diff:.2} AA")

## Show new structure

In [ ]:
molobj_new = ppqm.chembridge.axyzc_to_molobj(atoms, new_coordinates, charge)
ppqm_jupyter.show_molobj(molobj_new)